### Reference
* 李宏毅老師投影片
* Pytorch document

# Attention 說明

![](./image/seq2seq.png)

![](./image/atten1.png)

![](./image/attention.png)

# Pytorch Flow

![](./image/atten_flow.png)

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

In [18]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=5):
        super(AttnDecoderRNN, self).__init__()
        
        
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)
        
        
        
        #########################################
        # 第一步： calculate weights !
        # match fun. : Applies a linear transformation to the incoming data: y = Ax + b
        # self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        # using the decoder’s input and hidden state as inputs
        o = torch.cat((embedded[0], hidden[0]), 1)
        o = self.attn(o)
        attn_weights = F.softmax(o, dim=1)
                      
                      
                      
        
        # attn_weights = F.softmax(
        #                 self.attn(torch.cat((embedded[0], hidden[0]), 1)), 
        #                          dim=1)
        
        
        
        ##########################################
        # 第二步：encoder_outputs 和  softmax運算後的權重相乘。
        # torch.bmm: Performs a batch matrix-matrix product of matrices.
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))
        
        

        # 合成 Decoder input 
        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)
        
        
        #############################################
        # Training RNN
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

### 舉例：image caption
![](./image/image_caption.png)